# **XGBoost**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xgboost

In [3]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_1s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [6]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [7]:
train_data

{'melspectrogram': array([[[-2.10519694e-02, -2.64618150e-03, -1.09052584e-02, ...,
          -1.14497980e-02,  0.00000000e+00, -1.33858034e-02],
         [-1.99002311e-01, -2.20890388e-01, -2.76137829e-01, ...,
          -2.59782881e-01, -2.09464088e-01, -1.87755451e-01],
         [-2.39491284e-01, -2.51040578e-01, -2.89648235e-01, ...,
          -2.82749534e-01, -2.52160460e-01, -2.39700243e-01],
         ...,
         [-9.98796940e-01, -9.98772740e-01, -9.98712122e-01, ...,
          -9.96163070e-01, -9.97696698e-01, -9.98156428e-01],
         [-9.99429047e-01, -9.99434769e-01, -9.99466956e-01, ...,
          -9.99509513e-01, -9.99450743e-01, -9.99419093e-01],
         [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
          -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
 
        [[-1.80637967e-02, -3.36689858e-08, -7.33598135e-03, ...,
          -1.05369762e-02, -8.22041358e-04, -1.49706351e-02],
         [-1.93169340e-01, -2.11562306e-01, -2.58488506e-01, ...,
   

In [8]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [9]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [10]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [11]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [12]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [13]:
display(train_y.shape)
display(train_y[:15])

(12565,)

array([ 0, 19,  2,  1, 11, 17, 15,  2,  2,  2,  0,  1,  1, 12,  0],
      dtype=int32)

In [14]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(12565, 128, 87)

array([[-1.73581708e-02,  0.00000000e+00, -8.14180914e-03, ...,
        -9.85445548e-03, -1.71658248e-04, -1.32468548e-02],
       [-1.93985939e-01, -2.12422282e-01, -2.69561797e-01, ...,
        -2.80415505e-01, -2.17691883e-01, -1.94388315e-01],
       [-2.68526703e-01, -2.86887318e-01, -3.42721850e-01, ...,
        -3.53653818e-01, -2.91956156e-01, -2.68852562e-01],
       ...,
       [-9.99117315e-01, -9.99106705e-01, -9.99057472e-01, ...,
        -9.98603582e-01, -9.98687506e-01, -9.98692393e-01],
       [-9.99352336e-01, -9.99339044e-01, -9.99306679e-01, ...,
        -9.99599993e-01, -9.99699354e-01, -9.99706149e-01],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]])

'mfcc'

(12565, 20, 87)

...

(12565, 84, 87)

array([[-2.13950068e-01, -2.38894999e-01, -3.80664200e-01, ...,
        -1.41635209e-01, -1.01275146e-02, -1.78813934e-07],
       [-4.45840538e-01, -5.22175133e-01, -6.68615699e-01, ...,
        -4.22790468e-01, -2.85212219e-01, -2.24623069e-01],
       [-7.63046622e-01, -7.80026078e-01, -9.83664155e-01, ...,
        -7.36702800e-01, -5.25470257e-01, -5.25247097e-01],
       ...,
       [-2.00844555e+01, -7.19054260e+01, -6.65617599e+01, ...,
        -4.13212929e+01, -4.68384933e+01, -2.91117020e+01],
       [-2.03201313e+01, -6.45224457e+01, -6.51016922e+01, ...,
        -4.41159325e+01, -4.89345970e+01, -3.10661354e+01],
       [-2.05067978e+01, -6.17005310e+01, -5.89219780e+01, ...,
        -5.76679039e+01, -5.61718521e+01, -3.33402481e+01]])

'id'

(12565,)

'XC483906.mp3'

In [15]:
display(val_y.shape)
display(val_y[:15])

(3318,)

array([ 3,  2,  1,  0,  0,  8, 19,  2, 19,  5, 19, 15, 11,  7,  2],
      dtype=int32)

In [16]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(3318, 128, 87)

array([[-1.19495122e-02,  0.00000000e+00, -5.92639437e-03, ...,
        -9.61586274e-03, -1.45224054e-04, -1.35156866e-02],
       [-1.32104889e-01, -1.48279309e-01, -2.04118401e-01, ...,
        -2.71819055e-01, -2.12780029e-01, -1.90217406e-01],
       [-1.77966341e-01, -1.93037197e-01, -2.48268843e-01, ...,
        -3.41875643e-01, -2.82413810e-01, -2.60217577e-01],
       ...,
       [-8.56934249e-01, -8.21815670e-01, -8.13137054e-01, ...,
        -9.90914822e-01, -9.94373798e-01, -9.93971646e-01],
       [-9.33930933e-01, -9.18021381e-01, -9.22511458e-01, ...,
        -9.97301877e-01, -9.96644974e-01, -9.95461941e-01],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
        -1.00000000e+00, -9.97628629e-01, -9.96140599e-01]])

'mfcc'

(3318, 20, 87)

...

(3318, 84, 87)

array([[ -0.81876791,  -0.83223093,  -0.95862055, ...,  -1.04782999,
         -0.91178417,  -0.89874911],
       [ -1.06328416,  -1.15007341,  -1.30937183, ...,  -1.50782466,
         -1.36429203,  -1.29628861],
       [ -1.31883109,  -1.31582081,  -1.49792707, ...,  -2.05177641,
         -1.80057609,  -1.77171004],
       ...,
       [-18.08393478, -52.00915909, -46.19884491, ..., -56.79980087,
        -66.42909241, -28.33953667],
       [-17.75449371, -48.76427078, -46.56741714, ..., -68.17790222,
        -62.38970947, -32.60255051],
       [-17.46328926, -54.79916   , -53.8106575 , ..., -59.53269958,
        -60.99742126, -34.56028748]])

'id'

(3318,)

'XC428404.mp3'

## **Random Forest Model**

Saving evaluation results

In [22]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [23]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [24]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 21228)

In [25]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 21228)

Fit the model with training data

In [26]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [27]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6772151898734177


In [28]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.71      0.58      0.64       455
           1       0.42      0.64      0.51       492
           2       0.80      0.77      0.79       889
           3       0.91      0.95      0.93       150
           4       0.15      0.04      0.07        67
           5       0.45      0.35      0.39        43
           6       0.36      0.50      0.42        24
           7       1.00      0.91      0.95        44
           8       0.72      0.68      0.70        50
           9       0.96      0.93      0.95       169
          10       0.79      0.72      0.75        53
          11       0.74      0.39      0.51        66
          12       0.77      0.78      0.77        59
          13       0.55      0.70      0.62        57
          14       0.46      0.87      0.61        38
          15       0.56      0.37      0.45       311
          16       0.95      0.98      0.96   

### **With MFCCs only**

In [29]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 1740)

In [30]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 1740)

In [31]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [32]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6705846895720313


In [33]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       455
           1       0.45      0.71      0.55       492
           2       0.76      0.57      0.65       889
           3       0.88      0.96      0.92       150
           4       0.75      0.04      0.08        67
           5       0.44      0.33      0.37        43
           6       0.29      0.38      0.33        24
           7       0.83      0.89      0.86        44
           8       0.61      0.56      0.58        50
           9       0.95      0.93      0.94       169
          10       0.68      0.57      0.62        53
          11       0.57      0.32      0.41        66
          12       0.75      0.73      0.74        59
          13       0.57      0.61      0.59        57
          14       0.37      0.87      0.52        38
          15       0.62      0.51      0.56       311
          16       0.90      0.96      0.93   

### **With Mel-Spectrogram Only**

In [34]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 11136)

In [35]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 11136)

In [36]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [37]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6561181434599156


In [38]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.69      0.66      0.67       455
           1       0.46      0.63      0.53       492
           2       0.76      0.73      0.75       889
           3       0.88      0.99      0.93       150
           4       0.14      0.01      0.03        67
           5       0.05      0.02      0.03        43
           6       0.23      0.29      0.26        24
           7       0.80      0.89      0.84        44
           8       0.63      0.52      0.57        50
           9       0.92      0.90      0.91       169
          10       0.73      0.66      0.69        53
          11       0.57      0.32      0.41        66
          12       0.56      0.58      0.57        59
          13       0.51      0.61      0.56        57
          14       0.36      0.74      0.49        38
          15       0.58      0.42      0.49       311
          16       0.86      0.98      0.92   

## **With Chroma Only**

In [39]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 1044)

In [40]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 1044)

In [41]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [42]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.510548523206751


In [43]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.50      0.58      0.54       455
           1       0.29      0.43      0.35       492
           2       0.53      0.56      0.54       889
           3       0.62      0.60      0.61       150
           4       0.22      0.03      0.05        67
           5       0.53      0.47      0.49        43
           6       0.21      0.21      0.21        24
           7       0.94      0.73      0.82        44
           8       0.28      0.26      0.27        50
           9       0.74      0.82      0.78       169
          10       0.83      0.45      0.59        53
          11       0.36      0.14      0.20        66
          12       0.78      0.68      0.73        59
          13       0.49      0.49      0.49        57
          14       0.57      0.74      0.64        38
          15       0.62      0.44      0.52       311
          16       0.68      0.42      0.52   

## **With CQT Only**

In [44]:
training_features_3D = train_features['cqt']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 7308)

In [45]:
validation_features_3D = val_features['cqt']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 7308)

In [46]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [47]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5340566606389391


In [48]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.46      0.48       455
           1       0.37      0.68      0.48       492
           2       0.80      0.65      0.72       889
           3       0.63      0.61      0.62       150
           4       0.25      0.01      0.03        67
           5       0.33      0.05      0.08        43
           6       0.56      0.42      0.48        24
           7       0.98      0.91      0.94        44
           8       0.56      0.70      0.62        50
           9       0.68      0.61      0.64       169
          10       0.34      0.58      0.43        53
          11       0.66      0.35      0.46        66
          12       0.74      0.66      0.70        59
          13       0.43      0.47      0.45        57
          14       0.40      0.79      0.53        38
          15       0.15      0.13      0.14       311
          16       0.85      0.85      0.85   

## **With MFCCs and Mel-Spectrogram**

In [49]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 12876)

In [50]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 12876)

In [51]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [52]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6678722121760097


In [53]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.78      0.70      0.74       455
           1       0.43      0.57      0.49       492
           2       0.75      0.70      0.72       889
           3       0.93      0.96      0.94       150
           4       0.27      0.04      0.08        67
           5       0.19      0.12      0.14        43
           6       0.38      0.42      0.40        24
           7       0.80      0.91      0.85        44
           8       0.66      0.54      0.59        50
           9       0.97      0.92      0.95       169
          10       0.72      0.58      0.65        53
          11       0.64      0.41      0.50        66
          12       0.65      0.78      0.71        59
          13       0.50      0.63      0.56        57
          14       0.40      0.76      0.52        38
          15       0.56      0.50      0.53       311
          16       0.89      0.98      0.93   

## **With MFCCs and Chroma**

In [54]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 2784)

In [55]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 2784)

In [56]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [57]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6959011452682339


In [58]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       455
           1       0.44      0.64      0.52       492
           2       0.78      0.64      0.71       889
           3       0.88      0.95      0.91       150
           4       0.31      0.06      0.10        67
           5       0.48      0.28      0.35        43
           6       0.38      0.42      0.40        24
           7       0.98      0.93      0.95        44
           8       0.68      0.72      0.70        50
           9       0.95      0.93      0.94       169
          10       0.84      0.79      0.82        53
          11       0.71      0.52      0.60        66
          12       0.77      0.73      0.75        59
          13       0.56      0.63      0.60        57
          14       0.42      0.87      0.57        38
          15       0.68      0.56      0.61       311
          16       0.93      0.95      0.94   

## **With MFCCs and CQT**

In [59]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 9048)

In [60]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 9048)

In [61]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [62]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.646172393007836


In [63]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.74      0.54      0.62       455
           1       0.39      0.76      0.52       492
           2       0.83      0.63      0.72       889
           3       0.89      0.95      0.92       150
           4       0.21      0.04      0.07        67
           5       0.75      0.56      0.64        43
           6       0.31      0.46      0.37        24
           7       1.00      0.89      0.94        44
           8       0.67      0.68      0.67        50
           9       0.96      0.92      0.94       169
          10       0.70      0.70      0.70        53
          11       0.64      0.45      0.53        66
          12       0.89      0.71      0.79        59
          13       0.59      0.63      0.61        57
          14       0.44      0.84      0.58        38
          15       0.48      0.36      0.41       311
          16       0.93      0.91      0.92   

## **With Chroma and CQT**

In [64]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 8352)

In [65]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 8352)

In [66]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [67]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5991561181434599


In [68]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.45      0.48       455
           1       0.39      0.71      0.50       492
           2       0.79      0.72      0.75       889
           3       0.77      0.83      0.79       150
           4       0.00      0.00      0.00        67
           5       0.29      0.12      0.17        43
           6       0.58      0.46      0.51        24
           7       0.98      0.93      0.95        44
           8       0.60      0.62      0.61        50
           9       0.77      0.88      0.82       169
          10       0.46      0.70      0.55        53
          11       0.74      0.47      0.57        66
          12       0.82      0.56      0.67        59
          13       0.50      0.56      0.53        57
          14       0.53      0.84      0.65        38
          15       0.36      0.22      0.27       311
          16       0.89      0.87      0.88   

## **With Mel-Spectrogram and Chroma**

In [69]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 12180)

In [70]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 12180)

In [71]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [72]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6754068716094033


In [73]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.69      0.67      0.68       455
           1       0.48      0.64      0.55       492
           2       0.77      0.76      0.77       889
           3       0.87      0.99      0.92       150
           4       0.09      0.01      0.03        67
           5       0.07      0.02      0.04        43
           6       0.32      0.46      0.38        24
           7       0.93      0.89      0.91        44
           8       0.68      0.54      0.60        50
           9       0.89      0.95      0.92       169
          10       0.76      0.74      0.75        53
          11       0.67      0.39      0.50        66
          12       0.64      0.49      0.56        59
          13       0.52      0.60      0.56        57
          14       0.44      0.89      0.59        38
          15       0.60      0.37      0.46       311
          16       0.86      1.00      0.92   

## **With Mel-Spectrogram and CQT**

In [74]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 18444)

In [75]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 18444)

In [76]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [77]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6253767329716697


In [78]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.64      0.52      0.57       455
           1       0.39      0.60      0.47       492
           2       0.78      0.77      0.78       889
           3       0.89      0.98      0.93       150
           4       0.07      0.01      0.02        67
           5       0.00      0.00      0.00        43
           6       0.29      0.42      0.34        24
           7       0.98      0.93      0.95        44
           8       0.68      0.56      0.62        50
           9       0.92      0.91      0.91       169
          10       0.69      0.66      0.67        53
          11       0.47      0.21      0.29        66
          12       0.65      0.58      0.61        59
          13       0.52      0.60      0.56        57
          14       0.35      0.82      0.49        38
          15       0.42      0.27      0.33       311
          16       0.90      0.95      0.92   

## **With Mel-Spectrogram, MFCCs, CQT**

In [79]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 20184)

In [80]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 20184)

In [81]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [82]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6482820976491862


In [83]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.72      0.55      0.62       455
           1       0.40      0.62      0.49       492
           2       0.77      0.73      0.75       889
           3       0.93      0.95      0.94       150
           4       0.19      0.04      0.07        67
           5       0.38      0.26      0.31        43
           6       0.33      0.42      0.37        24
           7       1.00      0.93      0.96        44
           8       0.68      0.68      0.68        50
           9       0.99      0.95      0.97       169
          10       0.77      0.68      0.72        53
          11       0.55      0.32      0.40        66
          12       0.68      0.75      0.71        59
          13       0.50      0.58      0.54        57
          14       0.40      0.76      0.52        38
          15       0.46      0.33      0.39       311
          16       0.91      0.96      0.94   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [84]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 13920)

In [85]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 13920)

In [86]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [87]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6802290536467752


In [88]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.79      0.70      0.74       455
           1       0.43      0.62      0.51       492
           2       0.76      0.69      0.72       889
           3       0.91      0.95      0.93       150
           4       0.33      0.06      0.10        67
           5       0.47      0.16      0.24        43
           6       0.45      0.42      0.43        24
           7       0.75      0.89      0.81        44
           8       0.69      0.66      0.67        50
           9       0.95      0.95      0.95       169
          10       0.74      0.70      0.72        53
          11       0.78      0.47      0.58        66
          12       0.63      0.78      0.70        59
          13       0.52      0.65      0.58        57
          14       0.42      0.82      0.55        38
          15       0.62      0.52      0.56       311
          16       0.85      0.96      0.91   

# Review results from all models

In [89]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.677
3,chroma,1.0,0.511
8,chroma_cqt,1.0,0.599
4,cqt,1.0,0.534
2,melspectrogram,1.0,0.656
9,melspectrogram_chroma,1.0,0.675
12,melspectrogram_chroma_mfcc,1.0,0.680
10,melspectrogram_cqt,1.0,0.625
11,melspectrogram_cqt_mfcc,1.0,0.648
1,mfcc,1.0,0.671


In [90]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.676      0.927
3                    chroma     0.509      0.857
8                chroma_cqt     0.592      0.904
4                       cqt     0.532      0.880
2            melspectrogram     0.649      0.923
9     melspectrogram_chroma     0.665      0.928
10       melspectrogram_cqt     0.619      0.908
11  melspectrogram_cqt_mfcc     0.679      0.933
1                      mfcc     0.667      0.942
6               mfcc_chroma     0.696      0.947
7                  mfcc_cqt     0.651      0.932
5       mfcc_melspectrogram     0.666      0.929
